# Task II - Modeling Sellers, Demand Function, and Profit

This notebook creates hypothetical sellers, implements demand and profit functions based on price, advertising, and social influence.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 1. Load Cleaned Data

In [ ]:
data_path = 'cleaned_data.csv'
df = pd.read_csv(data_path)

print(f"Cleaned data shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

## 2. Identify Similar Products for Hypothetical Sellers

In [ ]:
def extract_product_category(description):
    if pd.isna(description):
        return 'Unknown'
    desc = str(description).upper()
    
    if 'CAKE CASE' in desc or 'CAKE' in desc:
        return 'Cake Cases'
    elif 'HEART' in desc or 'T-LIGHT' in desc or 'ORNAMENT' in desc:
        return 'Home Decor'
    elif 'PEN' in desc or 'PENCIL' in desc or 'MARKER' in desc:
        return 'Writing Supplies'
    elif 'MUG' in desc or 'CUP' in desc or 'PLATE' in desc or 'BOWL' in desc:
        return 'Tableware'
    elif 'DRAWER' in desc or 'CABINET' in desc or 'DRESSER' in desc or 'SIDEBOARD' in desc:
        return 'Furniture'
    elif 'BAG' in desc or 'PURSE' in desc:
        return 'Bags & Accessories'
    else:
        return 'Other'

df['Category'] = df['Description'].apply(extract_product_category)

print("Product Categories Distribution:")
print(df['Category'].value_counts())
print(f"\nCategory breakdown:")
for cat in df['Category'].unique():
    items = df[df['Category'] == cat]['StockCode'].nunique()
    print(f"  {cat}: {items} unique items")

## 3. Select High-Volume Category and Create Hypothetical Sellers

In [ ]:
selected_category = 'Cake Cases'
df_category = df[df['Category'] == selected_category].copy()

print(f"Selected category: {selected_category}")
print(f"Transactions: {len(df_category)}")
print(f"Unique items: {df_category['StockCode'].nunique()}")

price_groups = pd.cut(df_category['Price'], bins=3, labels=['Low', 'Medium', 'High'])
print(f"\nPrice distribution in {selected_category}:")
print(price_groups.value_counts())

print(f"\nTop 10 items in {selected_category}:")
top_items = df_category.groupby('StockCode').agg({
    'Description': 'first',
    'Price': 'mean',
    'Quantity': 'sum',
    'Invoice': 'count'
}).sort_values('Quantity', ascending=False).head(10)
print(top_items)

## 4. Create Hypothetical Sellers Based on Price Segmentation

In [ ]:
sellers_data = []

seller_1_items = df_category[df_category['Price'] <= df_category['Price'].quantile(0.33)]['StockCode'].unique()
seller_2_items = df_category[(df_category['Price'] > df_category['Price'].quantile(0.33)) & 
                               (df_category['Price'] <= df_category['Price'].quantile(0.67))]['StockCode'].unique()
seller_3_items = df_category[df_category['Price'] > df_category['Price'].quantile(0.67)]['StockCode'].unique()

sellers = {
    'Seller_A': seller_1_items,
    'Seller_B': seller_2_items,
    'Seller_C': seller_3_items
}

print("Hypothetical Sellers Distribution:")
for seller_name, items in sellers.items():
    seller_df = df_category[df_category['StockCode'].isin(items)]
    avg_price = seller_df['Price'].mean()
    total_quantity = seller_df['Quantity'].sum()
    transactions = len(seller_df)
    
    sellers_data.append({
        'Seller': seller_name,
        'Unique_Items': len(items),
        'Avg_Price': avg_price,
        'Total_Quantity': total_quantity,
        'Transactions': transactions,
        'Items': items
    })
    
    print(f"\n{seller_name}:")
    print(f"  Unique items: {len(items)}")
    print(f"  Avg price: ${avg_price:.2f}")
    print(f"  Total quantity sold: {total_quantity}")
    print(f"  Transactions: {transactions}")

sellers_df = pd.DataFrame(sellers_data)
print(f"\nSellers Summary:")
print(sellers_df[['Seller', 'Unique_Items', 'Avg_Price', 'Total_Quantity', 'Transactions']])

## 5. Define Seller Characteristics and Parameters

In [ ]:
class Seller:
    def __init__(self, name, price, cost, advertising_budget, base_demand, items_list):
        self.name = name
        self.price = price
        self.cost = cost
        self.advertising_budget = advertising_budget
        self.base_demand = base_demand
        self.items = items_list
        self.profit = 0
        self.demand = 0
    
    def __repr__(self):
        return f"Seller(name={self.name}, price={self.price}, cost={self.cost}, ad_budget={self.advertising_budget})"

seller_a = Seller(
    name='Seller_A',
    price=sellers_df[sellers_df['Seller'] == 'Seller_A']['Avg_Price'].values[0],
    cost=2.0,
    advertising_budget=100,
    base_demand=sellers_df[sellers_df['Seller'] == 'Seller_A']['Total_Quantity'].values[0] * 0.3,
    items_list=sellers['Seller_A']
)

seller_b = Seller(
    name='Seller_B',
    price=sellers_df[sellers_df['Seller'] == 'Seller_B']['Avg_Price'].values[0],
    cost=2.5,
    advertising_budget=150,
    base_demand=sellers_df[sellers_df['Seller'] == 'Seller_B']['Total_Quantity'].values[0] * 0.3,
    items_list=sellers['Seller_B']
)

seller_c = Seller(
    name='Seller_C',
    price=sellers_df[sellers_df['Seller'] == 'Seller_C']['Avg_Price'].values[0],
    cost=3.0,
    advertising_budget=120,
    base_demand=sellers_df[sellers_df['Seller'] == 'Seller_C']['Total_Quantity'].values[0] * 0.3,
    items_list=sellers['Seller_C']
)

sellers_list = [seller_a, seller_b, seller_c]

print("Initial Seller Configuration:")
for seller in sellers_list:
    print(f"\n{seller.name}:")
    print(f"  Price: ${seller.price:.2f}")
    print(f"  Production Cost: ${seller.cost:.2f}")
    print(f"  Advertising Budget: ${seller.advertising_budget:.2f}")
    print(f"  Base Demand: {seller.base_demand:.0f} units")
    print(f"  Profit Margin per unit: ${seller.price - seller.cost:.2f}")

## 6. Implement Demand Function

In [ ]:
def calculate_demand(seller_i, seller_j, alpha, beta, gamma, influence_score):
    D_i = (seller_i.base_demand + 
           alpha * seller_i.advertising_budget + 
           beta * (seller_i.price - seller_j.price) + 
           gamma * influence_score)
    
    return max(D_i, 0)

alpha = 0.05
beta = 15.0
gamma = 0.8
initial_influence = 100

print("Demand Function Parameters:")
print(f"  α (advertising impact): {alpha}")
print(f"  β (price sensitivity): {beta}")
print(f"  γ (social influence impact): {gamma}")
print(f"  Initial social influence score: {initial_influence}")

print("\n" + "="*60)
print("DEMAND CALCULATIONS (Initial State)")
print("="*60)

demands = {}
for i, seller_i in enumerate(sellers_list):
    for j, seller_j in enumerate(sellers_list):
        if i != j:
            D = calculate_demand(seller_i, seller_j, alpha, beta, gamma, initial_influence)
            demands[f"{seller_i.name}_vs_{seller_j.name}"] = D
            
print("\nDemand for each seller (considering each competitor):")
for seller_i in sellers_list:
    print(f"\n{seller_i.name}:")
    for seller_j in sellers_list:
        if seller_i.name != seller_j.name:
            demand_val = demands[f"{seller_i.name}_vs_{seller_j.name}"]
            print(f"  vs {seller_j.name}: {demand_val:.2f} units")

## 7. Implement Profit Function

In [ ]:
def calculate_profit(seller_i, seller_j, alpha, beta, gamma, influence_score):
    D_i = calculate_demand(seller_i, seller_j, alpha, beta, gamma, influence_score)
    seller_i.demand = D_i
    profit = (seller_i.price - seller_i.cost) * D_i - seller_i.advertising_budget
    seller_i.profit = profit
    return profit, D_i

print("Profit Calculations (Initial State):")
print("="*70)

profit_data = []

for i, seller_i in enumerate(sellers_list):
    for j, seller_j in enumerate(sellers_list):
        if i != j:
            profit, demand = calculate_profit(seller_i, seller_j, alpha, beta, gamma, initial_influence)
            
            profit_data.append({
                'Seller': seller_i.name,
                'Competitor': seller_j.name,
                'Price': seller_i.price,
                'Cost': seller_i.cost,
                'Margin': seller_i.price - seller_i.cost,
                'Demand': demand,
                'Ad_Budget': seller_i.advertising_budget,
                'Revenue': (seller_i.price - seller_i.cost) * demand,
                'Profit': profit
            })

profit_df = pd.DataFrame(profit_data)

for seller in sellers_list:
    seller_profits = profit_df[profit_df['Seller'] == seller.name]
    print(f"\n{seller.name}:")
    print(f"  Price: ${seller.price:.2f}")
    print(f"  Cost: ${seller.cost:.2f}")
    print(f"  Advertising Budget: ${seller.advertising_budget:.2f}")
    print(f"  Margin per unit: ${seller.price - seller.cost:.2f}")
    for idx, row in seller_profits.iterrows():
        print(f"  vs {row['Competitor']}:")
        print(f"    Demand: {row['Demand']:.2f} units")
        print(f"    Revenue: ${row['Revenue']:.2f}")
        print(f"    Profit: ${row['Profit']:.2f}")

print("\n" + "="*70)

## 8. Calculate and Store Initial Profits

In [ ]:
print("Initial Profit Summary (FIXED):")
print("="*70)

initial_profit_data = []
for seller in sellers_list:
    seller_profits = profit_df[profit_df['Seller'] == seller.name]
    avg_profit = seller_profits['Profit'].mean()
    total_demand = seller_profits['Demand'].sum()
    
    initial_profit_data.append({
        'Seller': seller.name,
        'Avg_Initial_Profit': avg_profit,
        'Avg_Initial_Demand': total_demand / len(seller_profits),
        'Initial_Price': seller.price,
        'Initial_Cost': seller.cost,
        'Initial_AD_Budget': seller.advertising_budget
    })
    
    print(f"\n{seller.name}:")
    print(f"  Average Profit (vs competitors): ${avg_profit:.2f}")
    print(f"  Average Demand: {total_demand / len(seller_profits):.2f} units")

initial_profit_df = pd.DataFrame(initial_profit_data)
print("\n" + "="*70)

## 9. Profit Analysis Summary

In [ ]:
print("Summary Statistics:")
print(f"\nTotal Demand across all scenarios: {profit_df['Demand'].sum():.2f} units")
print(f"Total Revenue (before ad costs): {profit_df['Revenue'].sum():.2f}")
print(f"Total Advertising Spend: {profit_df['Ad_Budget'].sum():.2f}")
print(f"Total Profit: {profit_df['Profit'].sum():.2f}")

print(f"\nAverage Profit per Seller Scenario: {profit_df['Profit'].mean():.2f}")
print(f"Best Profit Scenario: {profit_df['Profit'].max():.2f}")
print(f"Worst Profit Scenario: {profit_df['Profit'].min():.2f}")

print(f"\nProfit by Seller:")
for seller in sellers_list:
    seller_profits = profit_df[profit_df['Seller'] == seller.name]['Profit']
    avg_profit = seller_profits.mean()
    print(f"  {seller.name}: ${avg_profit:.2f} (avg across competitors)")

## 10. Sensitivity Analysis - Impact of Parameters

In [ ]:
print("Sensitivity Analysis: Impact of Advertising Budget")
print("="*70)

for multiplier in [0.5, 1.0, 1.5, 2.0]:
    print(f"\nAdvertising Budget Multiplier: {multiplier}x")
    total_profit = 0
    for i, seller_i in enumerate(sellers_list):
        temp_seller = Seller(
            name=seller_i.name,
            price=seller_i.price,
            cost=seller_i.cost,
            advertising_budget=seller_i.advertising_budget * multiplier,
            base_demand=seller_i.base_demand,
            items_list=seller_i.items
        )
        
        for j, seller_j in enumerate(sellers_list):
            if i != j:
                profit, demand = calculate_profit(temp_seller, seller_j, alpha, beta, gamma, initial_influence)
                total_profit += profit
    
    print(f"  Total Profit: ${total_profit:.2f}")

print("\n" + "="*70)
print("Sensitivity Analysis: Impact of Price Changes")
print("="*70)

for price_change in [-0.5, -0.25, 0, 0.25, 0.5]:
    print(f"\nPrice Change: {price_change:+.2f}")
    total_profit = 0
    for i, seller_i in enumerate(sellers_list):
        temp_seller = Seller(
            name=seller_i.name,
            price=seller_i.price + price_change,
            cost=seller_i.cost,
            advertising_budget=seller_i.advertising_budget,
            base_demand=seller_i.base_demand,
            items_list=seller_i.items
        )
        
        for j, seller_j in enumerate(sellers_list):
            if i != j:
                profit, demand = calculate_profit(temp_seller, seller_j, alpha, beta, gamma, initial_influence)
                total_profit += profit
    
    print(f"  Total Profit: ${total_profit:.2f}")

## 11. Visualization of Profit Landscape

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

ax = axes[0, 0]
seller_names = [s.name for s in sellers_list]
profits = []
for seller in sellers_list:
    seller_p = profit_df[profit_df['Seller'] == seller.name]['Profit'].mean()
    profits.append(seller_p)

ax.bar(seller_names, profits, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax.set_title('Average Profit by Seller', fontsize=12, fontweight='bold')
ax.set_ylabel('Profit ($)')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(profits):
    ax.text(i, v + 50, f'${v:.0f}', ha='center', fontweight='bold')

ax = axes[0, 1]
demands = profit_df.groupby('Seller')['Demand'].mean()
ax.bar(seller_names, demands, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax.set_title('Average Demand by Seller', fontsize=12, fontweight='bold')
ax.set_ylabel('Demand (units)')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(demands):
    ax.text(i, v + 5, f'{v:.0f}', ha='center', fontweight='bold')

ax = axes[1, 0]
prices = [s.price for s in sellers_list]
ax.bar(seller_names, prices, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax.set_title('Selling Price by Seller', fontsize=12, fontweight='bold')
ax.set_ylabel('Price ($)')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(prices):
    ax.text(i, v + 0.02, f'${v:.2f}', ha='center', fontweight='bold')

ax = axes[1, 1]
ad_budgets = [s.advertising_budget for s in sellers_list]
ax.bar(seller_names, ad_budgets, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax.set_title('Advertising Budget by Seller', fontsize=12, fontweight='bold')
ax.set_ylabel('Ad Budget ($)')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(ad_budgets):
    ax.text(i, v + 3, f'${v:.0f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('task_II_sellers_comparison.png', dpi=100, bbox_inches='tight')
plt.show()

print("Chart saved as 'task_II_sellers_comparison.png'")

## 12. Save All Configuration Files

In [ ]:
seller_config = []
for seller in sellers_list:
    seller_config.append({
        'Seller_Name': seller.name,
        'Price': seller.price,
        'Cost': seller.cost,
        'Advertising_Budget': seller.advertising_budget,
        'Base_Demand': seller.base_demand,
        'Profit_Margin': seller.price - seller.cost,
        'Num_Items': len(seller.items)
    })

config_df = pd.DataFrame(seller_config)
config_df.to_csv('seller_config.csv', index=False)
print("Seller configuration saved to 'seller_config.csv'")
print(f"\nConfiguration Summary:")
print(config_df.to_string(index=False))

profit_df.to_csv('initial_profit_calculations.csv', index=False)
print("\nInitial profit calculations saved to 'initial_profit_calculations.csv'")

initial_profit_df.to_csv('initial_profit_summary.csv', index=False)
print("\nInitial profit summary saved to 'initial_profit_summary.csv'")
print("\nInitial Profit Summary:")
print(initial_profit_df.to_string(index=False))